In [6]:
# Load packages
import s3fs; import xarray as xr; import numpy as np
import pandas as pd; import dask.array as da; import ocetrac
import matplotlib.pyplot as plt; import cartopy.crs as ccrs
import warnings; import expectexception
warnings.filterwarnings('ignore')
import netCDF4 as nc; import datetime as dt; import scipy
import intake; import pprint
# Allow multiple lines per cell to be displayed without print (default is just last line)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Enable more explicit control of DataFrame display (e.g., to omit annoying line numbers)
from IPython.display import HTML

In [7]:
# load my functions
from functions import *

In [8]:
# for magic functions
%load_ext memory_profiler
%load_ext line_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [34]:
# in order to run use apply_ocetrac_to_CESM2LE, we need this cell first
cat_url_orig = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)
subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical')
member_id_list = subset.df.member_id.unique()

In [8]:
%%time
ensemble_member_val = 0
radius_val = 3

detrended,blobs = apply_ocetrac_to_CESM2LE(ensemble_member_val, 0.9, radius_val, 0.75, start_val=0, end_val=1980) # ~ 6.5 minutes per ensemble member
SSTA_and_events = create_events_file(detrended, blobs)
SSTA_and_events.to_netcdf('SSTA_and_events_{}_{}.nc'.format(ensemble_member_val, radius_val))


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


minimum area: 349.0
inital objects identified 	 21386
final objects tracked 	 1203
CPU times: user 5min 40s, sys: 53 s, total: 6min 33s
Wall time: 6min 41s


In [73]:
%%time
# load in nino indices
nino34_first50 = xr.open_dataset('/glade/u/home/cassiacai/marine_heatwaves/data/nino34_first50.nc')
nino34_last50 = xr.open_dataset('/glade/u/home/cassiacai/marine_heatwaves/data/nino34_last50.nc')

nino4_first50 = xr.open_dataset('/glade/u/home/cassiacai/marine_heatwaves/data/nino4_first50.nc')
nino4_last50 = xr.open_dataset('/glade/u/home/cassiacai/marine_heatwaves/data/nino4_last50.nc')

# rename nino file variables
nino34_first50['nino34_ind'] = nino34_first50['SST']
nino34_first50 = nino34_first50.drop(['SST'])

nino4_first50['nino4_ind'] = nino4_first50['SST']
nino4_first50 = nino4_first50.drop(['SST'])

# nino indices for our specific ensemble member
nino34_ens0 = nino34_first50.nino34_ind[ensemble_member_val,:]
nino4_ens0 = nino4_first50.nino4_ind[ensemble_member_val,:]

CPU times: user 42.5 ms, sys: 955 µs, total: 43.5 ms
Wall time: 46.3 ms


In [9]:
%%time
# load our land mask file
land_mask = np.load('/glade/u/home/cassiacai/marine_heatwaves/data/SST_land.npy')
land_mask[land_mask > 0] = np.nan
land_mask[land_mask == 0.] = 1

CPU times: user 1.62 ms, sys: 0 ns, total: 1.62 ms
Wall time: 9.29 ms


In [12]:
SSTA_and_events =  xr.open_dataset('SSTA_and_events_0_3.nc')
total_no_mhws = number_of_mhws(SSTA_and_events)
print('There are', total_no_mhws, 'mhws in this ensemble member!')

There are 1203 mhws in this ensemble member!


In [17]:
# on one marine heatwave
mhw_id = 35
event_file = SSTA_and_events

duration = calc_duration(event_file, mhw_id)
cumulative_intensity, cumulative_intensity_monthly = calc_cumulativeintensity(event_file, mhw_id)
mean_intensity, mean_intensity_monthly = calc_meanintensity(event_file, mhw_id)
max_intensity, max_intensity_monthly = calc_maximumintensity(event_file, mhw_id)
std_intensity, std_intensity_monthly = calc_stdintensity(event_file, mhw_id)
coords_full, spatial_extents, max_spatial_extent, max_spatial_extent_time, mean_spatial_extent, cumulative_spatial_extent = calc_spatialextent(event_file, mhw_id)
perimeters = calc_perimeter(event_file, mhw_id)
percperivarea = calc_percperimetervsarea(spatial_extents, perimeters)
perc_sharedarea_ls = calc_compltodeform(coords_full, spatial_extents)
deform = calc_deform(perc_sharedarea_ls)

In [85]:
######## Not necessary to run this cell
# previously 

# took about 36 minutes
initialization_SSTA_ar = np.zeros((no_mhws,192,288))
duration_ar = np.zeros((no_mhws))
initialization_when_ar = np.zeros((no_mhws))

for mhw_id in range(1,no_mhws): # no_mhws  
    for_one_mhw = SSTA_and_events.where(SSTA_and_events.labels==mhw_id, drop=False)
    
    
    mhw_when = np.argwhere(for_one_mhw.labels.max(axis=(1,2)).values > 0.)
    first_timestep = mhw_when[0][0]
    initialization_when_ar[mhw_id] = first_timestep
    initialization_SSTA_ar[mhw_id,:,:] = for_one_mhw.SSTA[first_timestep,:,:].values
    
    duration = calc_duration(SSTA_and_events, mhw_id)
    duration_ar[mhw_id] = duration
    
# np.save('initialization_SSTA_ar_0.npy', initialization_SSTA_ar)    
# initialization_SSTA_ar_0 = np.load('initialization_SSTA_ar_0.npy')

# np.save('duration_ar_0.npy', duration_ar)    
# duration_ar_0 = np.load('duration_ar_0.npy')

# np.save('initialization_when_ar_0.npy', initialization_when_ar)    
# initialization_when_ar_0 = np.load('initialization_when_ar_0.npy')

initialization_SSTA_ar_0 = np.load('initialization_SSTA_ar_0.npy')
duration_ar_0 = np.load('duration_ar_0.npy')
initialization_when_ar_0 = np.load('initialization_when_ar_0.npy')

In [15]:
# functions
def number_of_mhws(event_file):
    return len(np.unique(event_file.labels)) - 1

def calc_duration(event_file, mhw_id):
    return len(event_file.where(event_file.labels==mhw_id, drop=True).time)

def calc_cumulativeintensity(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=True)
    cumulative_intensity = np.nansum(for_one_mhw.SSTA)
    cumulative_intensity_monthly = for_one_mhw.SSTA.sum(axis=(1,2)).values
    return cumulative_intensity, cumulative_intensity_monthly

def calc_meanintensity(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=True)
    mean_intensity = np.nanmean(for_one_mhw.SSTA)
    mean_intensity_monthly = for_one_mhw.SSTA.mean(axis=(1,2)).values
    return mean_intensity, mean_intensity_monthly

def calc_maximumintensity(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=True)
    max_intensity = np.nanmax(for_one_mhw.SSTA)
    max_intensity_monthly = for_one_mhw.SSTA.max(axis=(1,2)).values
    return max_intensity, max_intensity_monthly

def calc_stdintensity(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=True)
    std_intensity = np.nanstd(for_one_mhw.SSTA)
    std_intensity_monthly = for_one_mhw.SSTA.std(axis=(1,2)).values
    return std_intensity, std_intensity_monthly

def calc_spatialextent(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=True)
    spatial_extents = []
    coords_full = []
    for i in range(len(for_one_mhw.time)):
        for_onetimestep_stacked = for_one_mhw.labels[i,:,:].stack(zipcoords=['lat','lon'])
        intermed = for_onetimestep_stacked[for_onetimestep_stacked.notnull()].zipcoords.values
        lats = [x[0] for x in intermed]; lons = [x[1] for x in intermed]
        coords = list(zip(lats, lons))
        coords_full.append(coords)
        y,x=zip(*coords)
        dlon = [np.cos(y[c]*np.pi/180)*(111.320*1) for c in np.arange(0, len(coords))]; dlat = (110.574 *1) * np.ones(len(dlon))
        area = np.sum(dlon*dlat)
        spatial_extents.append(area)
    max_spatial_extent = np.max(spatial_extents)
    max_spatial_extent_time = np.argmax(spatial_extents)
    mean_spatial_extent = np.mean(spatial_extents)
    cumulative_spatial_extent = np.sum(spatial_extents)
    return coords_full, spatial_extents, max_spatial_extent, max_spatial_extent_time, mean_spatial_extent, cumulative_spatial_extent

def initialization(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=False)
    mhw_when = np.argwhere(for_one_mhw.labels.max(axis=(1,2)).values > 0.)
    first_timestep = mhw_when[0][0]
    bymonth = np.resize(np.arange(1,13),12*166)[1:-11]
    month = bymonth[first_timestep]
    return first_timestep, for_one_mhw.SSTA[first_timestep,:,:].values, month

from skimage.measure import find_contours
from haversine import haversine, Unit
from scipy.interpolate import interp1d

def calc_perimeter(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=False)
    first_timestep, first_array, month = initialization(event_file, mhw_id)
    timesteps_to_choose_from = np.arange(first_timestep, first_timestep+duration)

    convert_long_range = interp1d([0,360],[-180,180])
    perimeter_ls = []
    for i in timesteps_to_choose_from:
        bw = for_one_mhw.labels[i,:,:].values > 0
        contours = find_contours(bw)
        distance_ls = []
        for contour_num in range(len(contours)):
            latitudes = for_one_mhw.lat.values[contours[contour_num][:,0].astype(int)]
            longitudes = for_one_mhw.lon.values[contours[contour_num][:,1].astype(int)]    
            coords = list(zip(latitudes, convert_long_range(longitudes)))

            for i in range(len(coords)-1):
                distance = haversine(coords[i], coords[i+1],Unit.KILOMETERS)
                distance_ls.append(distance)
            distance_ls.append(haversine(coords[len(coords)-1], coords[0],Unit.KILOMETERS))
        perimeter = np.sum(distance_ls)
        perimeter_ls.append(perimeter)
    return perimeter_ls  

def calc_percperimetervsarea(spatial_extents, perimeters):
    return (np.asarray(perimeters)/np.asarray(spatial_extents))*100

def convert_from_timeres_to_months(time_step):
    bymonth = np.resize(np.arange(1,13),12*166)[1:-11]
    month = bymonth[first_timestep]
    return month

def calc_compltodeform(coords_full, spatial_extents):
    perc_sharedarea_ls = []
    for i in range(len(coords_full)-1):
        a_set = set(coords_full[i])
        b_set = set(coords_full[i+1])
        if a_set & b_set:
            coords = a_set & b_set
            y,x=zip(*coords)
            dlon = [np.cos(y[c]*np.pi/180)*(111.320*1) for c in np.arange(0, len(coords))]; dlat = (110.574 *1) * np.ones(len(dlon))
            sharedarea = np.sum(dlon*dlat)
            perc_sharedarea_ls.append((sharedarea/ (spatial_extents[i] + spatial_extents[i+1]))*100)
        else:
            sharedareaarea = 0
            perc_sharedarea_ls.append((sharedarea/ (spatial_extents[i] + spatial_extents[i+1]))*100)
    return perc_sharedarea_ls

def calc_deform(perc_sharedarea_ls):
    return np.asarray(100 - np.asarray(perc_sharedarea_ls))

def calc_whenlargesmall(spatial_extents):
    when_large = (np.argmax(spatial_extents) / len(spatial_extents))*100
    when_small = (np.argmin(spatial_extents) / len(spatial_extents))*100
    return when_large, when_small

def cross_correlation_spat(event_file, mhw_id):
    for_one_mhw = event_file.where(event_file.labels==mhw_id, drop=False)
    first_timestep, first_array, month = initialization(event_file, mhw_id)
    timesteps_to_choose_from = np.arange(first_timestep, first_timestep+duration)
    cc_image_array = np.zeros((len(timesteps_to_choose_from), 192,288))    

    for i in range(len(timesteps_to_choose_from[:-1])):
        image = for_one_mhw.SSTA[timesteps_to_choose_from[i],:,:].values
        image = np.nan_to_num(image)
        offset_image = for_one_mhw.SSTA[timesteps_to_choose_from[i+1],:,:].values
        offset_image = np.nan_to_num(offset_image)
        image_product = np.fft.fft2(image) * np.fft.fft2(offset_image).conj()
        cc_image = np.fft.fftshift(np.fft.ifft2(image_product))
        cc_image_array[i,:,:] = np.real(cc_image)
    return cc_image_array